In [2]:
import logging
import h5py
import numpy as np
import warnings
from SMOPCA import SMOPCA, utils
from sklearn.cluster import KMeans

for handler in logging.root.handlers[:]:  # avoid DEBUG level information in jupyter notebook
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO)  # use DEBUG for verbose information
warnings.filterwarnings('ignore')

data_file = h5py.File("../data/RealData/CITEseq/SLN111D1.h5", 'r')
X1 = np.array(data_file['X1'])  # gene
X2 = np.array(data_file['X2'])  # protein
pos = np.array(data_file['pos'])  # umap pos
y = np.array(data_file['Y'])
data_file.close()

In [3]:
X1, X2 = utils.preprocess_hvg(x_list=[X1, X2], select_list=[True, False], top=1000)  # select hvg for genes only, considering the number of proteins
smopca = SMOPCA(Y_list=[X1.T, X2.T], Z_dim=20)
smopca.buildKernel(pos=pos, kernel_type='matern', length_scale=0.25)
smopca.estimateSigmaW(sigma_init_list=(1, 1), tol=2e-5, sigma_xtol_list=(1e-6, 1e-6))
z = smopca.calculatePosterior()
y_pred = KMeans(n_clusters=len(np.unique(y)), n_init=100).fit_predict(z.T)
ami, nmi, ari = utils.clustering_metric(y, y_pred)
print("AMI={:.4f}, NMI={:.4f}, ARI={:.4f}".format(ami, nmi, ari))

selecting top 1000 hvg for modality 1
Chosen offset: 0.26
normalizing counts


INFO:SMOPCA:SMOPCA object created, with 8853 cells and [1000, 110] features
INFO:SMOPCA:calculating matern kernel, nu = 1.5, length_scale = 0.25
INFO:SMOPCA:start estimating parameters, this will take a while...
INFO:SMOPCA:estimating sigma1
INFO:SMOPCA:sigma1 using bound: (0.70000, 0.80000)
INFO:SMOPCA:iter 0 sigma1 brentq done, sigma1sqr = 1.00000, sigma1hatsqr = 0.72547
INFO:SMOPCA:iter 1 sigma1 brentq done, sigma1sqr = 0.72547, sigma1hatsqr = 0.72538
INFO:SMOPCA:iter 2 sigma1 brentq done, sigma1sqr = 0.72538, sigma1hatsqr = 0.72538
INFO:SMOPCA:reach tolerance threshold, sigma1 done!
INFO:SMOPCA:estimating sigma2
INFO:SMOPCA:sigma2 using bound: (0.60000, 0.70000)
INFO:SMOPCA:iter 0 sigma2 brentq done, sigma2sqr = 1.00000, sigma2hatsqr = 0.66561
INFO:SMOPCA:iter 1 sigma2 brentq done, sigma2sqr = 0.66561, sigma2hatsqr = 0.66545
INFO:SMOPCA:iter 2 sigma2 brentq done, sigma2sqr = 0.66545, sigma2hatsqr = 0.66545
INFO:SMOPCA:reach tolerance threshold, sigma2 done!
INFO:SMOPCA:estimation c

AMI=0.7210, NMI=0.7255, ARI=0.6291
